In [2]:
import geopy
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

## 1.Setting up the raw data from Calgary

In [28]:
calgary_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T'
html_data = requests.get(calgary_url).text
soup = BeautifulSoup(html_data,"html5lib") 

In [29]:
calgary_df_content = []
table = soup.find("table")
for row in table.find_all("td") :
    cell = {}
    if row.span.text=='Not assigned':
        pass
    
    else:
        borough = row.span.text.split('(')[0]
        
        if borough == 'Calgary':
            cell['PostalCode'] = row.text[:3]
            cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
            calgary_df_content.append(cell)
            
        else:
            pass

In [30]:
calgary_df1=pd.DataFrame(data=calgary_df_content)
print(calgary_df1.shape)
calgary_df1.head()

(31, 2)


,PostalCode,Neighborhood
0,T2A,"Penbrooke Meadows, Marlborough"
1,T3A,"Dalhousie, Edgemont, Hamptons, Hidden Valley"
2,T2B,"Forest Lawn, Dover, Erin Woods"
3,T3B,"Montgomery, Bowness, Silver Springs, Greenwood"
4,T2C,"Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains"


In [31]:
# Had to find coordinates with https://www.latlong.net/
calgary_coordinates_url ='https://raw.githubusercontent.com/Armando12pdf/Coursera_Capstone/main/Calgary%20coordinates.csv'
calgary_coordinates_df=pd.read_csv(calgary_coordinates_url)
print(calgary_coordinates_df.shape)
calgary_coordinates_df.head()

(31, 3)


,Postal code,Latitude,Longitude
0,T2A,51.04968,-113.96432
1,T3A,51.12454,-114.14289
2,T2B,51.02533,-113.97890
3,T3B,51.08963,-114.19751
4,T2C,50.98122,-113.99786


In [32]:
#Concatenate with respective latitude and longitude
calgary_df1['Latitude'] = calgary_df1.PostalCode.map(calgary_coordinates_df.set_index('Postal code')['Latitude'].to_dict())
calgary_df1['Longitude'] = calgary_df1.PostalCode.map(calgary_coordinates_df.set_index('Postal code')['Longitude'].to_dict())
print(calgary_df1.shape)
calgary_df1.head()

(31, 4)


,PostalCode,Neighborhood,Latitude,Longitude
0,T2A,"Penbrooke Meadows, Marlborough",51.04968,-113.96432
1,T3A,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289
2,T2B,"Forest Lawn, Dover, Erin Woods",51.02533,-113.97890
3,T3B,"Montgomery, Bowness, Silver Springs, Greenwood",51.08963,-114.19751
4,T2C,"Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains",50.98122,-113.99786


In [33]:
#Define foursquare credentials
CLIENT_ID = 'ZZBEKKTJGWTU0HNGJ2ROO5JDHSKAXVABHUJKBBYZG1WNWQ0W' # your Foursquare ID
CLIENT_SECRET = 'JS1SDJDRRFBWX0SXVSS5GPCXVG4J3BAKY0AWJMLE2IJH4OFG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [34]:
#Function that gets venues

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
#Get venues for calgary neighborhoods
calgary_venues = getNearbyVenues(names=calgary_df1['Neighborhood'],
                                   latitudes=calgary_df1['Latitude'],
                                   longitudes=calgary_df1['Longitude'])
calgary_venues.head()

Penbrooke Meadows, Marlborough
Dalhousie, Edgemont, Hamptons, Hidden Valley
Forest Lawn, Dover, Erin Woods
Montgomery, Bowness, Silver Springs, Greenwood
Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains
Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta
Bridgeland, Greenview, Zoo, YYC
Lakeview, Glendale, Killarney, Glamorgan
Inglewood, Burnsland, Chinatown, East Victoria Park, Saddledome
Hawkwood, Arbour Lake, Citadel, Ranchlands, Royal Oak, Rocky Ridge
Highfield, Burns Industrial
Discovery Ridge, Signal Hill, West Springs,Christie Park, Patterson, Cougar Ridge
Queensland, Lake Bonavista, Willow Park, Acadia
Martindale, Taradale, Falconridge, Saddle Ridge
Thorncliffe, Tuxedo Park
Sandstone, MacEwan Glen, Beddington, Harvest Hills, Coventry Hills, Panorama Hills
Brentwood, Collingwood, Nose Hill
Tuscany, Scenic Acres
Mount Pleasant, Capitol Hill, Banff Trail
Cranston, Auburn Bay, Mahogany
Kensington, Westmont, Parkdale, University
City Centre, Calgary Tower
Symons Valley
Conn

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289,Edgemont Construction Ltd,51.123928,-114.144313,Construction & Landscaping
1,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289,Edgemont City,51.126473,-114.138997,Asian Restaurant
2,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289,Friends Cappuccino Bar & Bake Shop,51.126370,-114.138676,Café
3,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289,On the Rocks,51.126893,-114.139355,Wine Shop
4,"Forest Lawn, Dover, Erin Woods",51.02533,-113.97890,7-Eleven,51.026558,-113.980414,Convenience Store


In [36]:
#One hot encoding of calgary venues
calgary_onehot_df = pd.get_dummies(calgary_venues[['Venue Category']], prefix="", prefix_sep="")
calgary_onehot_df.insert(loc=0, column='Neighborhood name', value=calgary_venues['Neighborhood'].values.tolist()) 
print(calgary_onehot_df.shape)
calgary_onehot_df.head()

(237, 102)


,Neighborhood name,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Café,Camera Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Classroom,Construction & Landscaping,Convenience Store,Department Store,Diner,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Elementary School,Fast Food Restaurant,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Garden,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym / Fitness Center,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Insurance Office,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Music Store,New American Restaurant,Other Repair Shop,Park,Pharmacy,Pier,Pizza Place,Playground,Poutine Place,Professional & Other Places,Pub,Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shopping Mall,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Vietnamese Restaurant,Water Park,Wine Shop,Yoga Studio
0,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,"Forest Lawn, Dover, Erin Woods",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
#Regroup calgary venues. This will be the data used with the ML models
calgary_grouped_df = calgary_onehot_df.groupby('Neighborhood name').mean().reset_index()
print(calgary_grouped_df.shape)
calgary_grouped_df.head()

(30, 102)


,Neighborhood name,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Beer Bar,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Bus Station,Business Service,Café,Camera Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Classroom,Construction & Landscaping,Convenience Store,Department Store,Diner,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Elementary School,Fast Food Restaurant,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Garden,Gas Station,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym / Fitness Center,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Inn,Insurance Office,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Moroccan Restaurant,Music Store,New American Restaurant,Other Repair Shop,Park,Pharmacy,Pier,Pizza Place,Playground,Poutine Place,Professional & Other Places,Pub,Restaurant,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shopping Mall,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Vietnamese Restaurant,Water Park,Wine Shop,Yoga Studio
0,"Braeside, Cedarbrae, Woodbine",0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.333333,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.333333,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"Brentwood, Collingwood, Nose Hill",0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,1.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"Bridgeland, Greenview, Zoo, YYC",0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.200000,0.0,0.000000,0.2,0.200000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.2,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.2,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
3,"City Centre, Calgary Tower",0.0,0.0,0.0,0.0,0.023256,0.023256,0.069767,0.000000,0.023256,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.046512,0.000000,0.0,0.000000,0.0,0.023256,0.162791,0.0,0.000000,0.0,0.023256,0.000000,0.0,0.0,0.000000,0.0,0.023256,0.0,0.023256,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.023256,0.0

In [55]:
# Some neighborhoods didn't get results, so they will be removed from our initial df. 
print(len(set(calgary_grouped_df['Neighborhood name'])))
print(len(set(calgary_df1['Neighborhood'])))

30
30


In [56]:
# Remove the neighborhoods in original df for which the API did provide information. 
for element in calgary_df1['Neighborhood']:
    if element in set(calgary_grouped_df['Neighborhood name']):
        pass
        
    else:
        print(element)
        calgary_df1 = calgary_df1[calgary_df1.Neighborhood != element]

print(len(set(calgary_grouped_df['Neighborhood name'])))        
print(len(calgary_df1['Neighborhood']))

30
30


## 2.Positive data learning with SVM

### 2.1 Prepare positively labeled raw data

In [41]:
#Postal codes were gathered manually. Their latitudes and longitudes were obtained from https://www.latlong.net/
positive_coordinates_url ='https://raw.githubusercontent.com/Armando12pdf/Coursera_Capstone/main/Positive%20locations%20coordinates.csv'
example_labs_df=pd.read_csv(positive_coordinates_url)
example_labs_df.head()

,Postal code,Latitude,Longitude
0,G1C,46.881771,-71.189369
1,G1E,46.860130,-71.194054
2,G1M,46.817230,-71.269836
3,G6W,46.757560,-71.225570
4,H1K,45.608180,-73.544520


In [42]:
#Add a neighborhood column
labs_labels =[]
for number in range(1,(example_labs_df.shape[0]+1)) :
    new_lab_number = 'example lab ' + str(number)
    labs_labels.append(new_lab_number)

example_labs_df.insert(loc=1, column='Neighborhood', value=labs_labels) 

In [43]:
example_labs_df.head()

,Postal code,Neighborhood,Latitude,Longitude
0,G1C,example lab 1,46.881771,-71.189369
1,G1E,example lab 2,46.860130,-71.194054
2,G1M,example lab 3,46.817230,-71.269836
3,G6W,example lab 4,46.757560,-71.225570
4,H1K,example lab 5,45.608180,-73.544520


In [44]:
#Get venues for the labs used as positive examples
example_labs_venues = getNearbyVenues(names=example_labs_df['Neighborhood'],
                                   latitudes=example_labs_df['Latitude'],
                                   longitudes=example_labs_df['Longitude'])

example lab 1
example lab 2
example lab 3
example lab 4
example lab 5
example lab 6
example lab 7
example lab 8
example lab 9
example lab 10
example lab 11
example lab 12
example lab 13
example lab 14
example lab 15
example lab 16
example lab 17
example lab 18
example lab 19
example lab 20
example lab 21
example lab 22
example lab 23
example lab 24
example lab 25
example lab 26
example lab 27
example lab 28
example lab 29
example lab 30
example lab 31
example lab 32
example lab 33
example lab 34
example lab 35
example lab 36
example lab 37
example lab 38
example lab 39
example lab 40
example lab 41
example lab 42
example lab 43
example lab 44
example lab 45
example lab 46
example lab 47
example lab 48
example lab 49
example lab 50
example lab 51
example lab 52
example lab 53
example lab 54
example lab 55
example lab 56
example lab 57
example lab 58
example lab 59
example lab 60
example lab 61
example lab 62
example lab 63
example lab 64
example lab 65
example lab 66
example lab 67
exam

In [45]:
example_labs_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,example lab 1,46.881771,-71.189369,Lou-Tec Québec (Beauport),46.882766,-71.185699,Rental Service
1,example lab 1,46.881771,-71.189369,Pneus Ratté,46.881937,-71.185247,Auto Garage
2,example lab 1,46.881771,-71.189369,Broue Resto,46.879427,-71.184905,American Restaurant
3,example lab 1,46.881771,-71.189369,Groupe Diane Lapointe Designers,46.884144,-71.184635,Construction & Landscaping
4,example lab 2,46.860130,-71.194054,Senor Sombrero,46.861217,-71.188514,Mexican Restaurant


In [46]:
#One hot encoding of the example labs
labs_onehot_df = pd.get_dummies(example_labs_venues[['Venue Category']], prefix="", prefix_sep="")
labs_onehot_df.insert(loc=0, column='Neighborhood name', value=example_labs_venues['Neighborhood'].values.tolist()) 
print(labs_onehot_df.shape)
labs_onehot_df.head()

(959, 213)


,Neighborhood name,Adult Boutique,Airport,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,City,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hockey Arena,Hockey Rink,Home Service,Hostel,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Liquor Store,Luggage Store,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Noodle House,Office,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Supply Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train,Train Station,Tree,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,example lab 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,example lab 1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,example lab 1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [53]:
#Regroup the venues of the example labs
labs_grouped_df = labs_onehot_df.groupby('Neighborhood name').mean().reset_index()
print(labs_grouped_df.shape)
labs_grouped_df.head()

(96, 213)


,Neighborhood name,Adult Boutique,Airport,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,City,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hockey Arena,Hockey Rink,Home Service,Hostel,Hotel,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Liquor Store,Luggage Store,Massage Studio,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Noodle House,Office,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Supply Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train,Train Station,Tree,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,example lab 1,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,example lab 10,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0

### 2.2 Train and evaluate the one-class SVM model

In [48]:
# When training a model, the features of the training data set have to match those of the target data set
# They do not match in this situation:
print('features in labs: ' + str(len(labs_grouped_df.columns)))
print('features in calgary: ' + str(len(calgary_grouped_df.columns)))

features in labs: 213
features in calgary: 102


In [49]:
#Columns in the training set that are not within the target data set, will be removed
for element in labs_grouped_df.columns.tolist():
    if element not in calgary_grouped_df.columns.tolist():
        labs_grouped_df.drop(columns=[element], inplace=True)
        
    else:
        pass

print('features in labs: ' + str(len(labs_grouped_df.columns)))
print('features in calgary: ' + str(len(calgary_grouped_df.columns)))

features in labs: 78
features in calgary: 102


In [50]:
# Repeat in other direction for features to match
# Columns in the training set that are not within the target data set, will be removed
for element in calgary_grouped_df.columns.tolist():
    if element not in labs_grouped_df.columns.tolist():
        calgary_grouped_df.drop(columns=[element], inplace=True)
        
    else:
        pass

print('features in labs: ' + str(len(labs_grouped_df.columns)))
print('features in calgary: ' + str(len(calgary_grouped_df.columns)))


features in labs: 78
features in calgary: 78


In [105]:
#Now we're ready to train our model
from sklearn.svm import OneClassSVM

In [106]:
#Split the data. 
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(labs_grouped_df, test_size=0.15, random_state=4)

In [107]:
#Fit
One_class_SVM = OneClassSVM(gamma='auto').fit(X_train.loc[:, X_train.columns != 'Neighborhood name'])

In [108]:
#Test
One_c_SVM_test=One_class_SVM.predict(X_test.loc[:, X_test.columns != 'Neighborhood name'])

In [109]:
#Test how many get labelled as positive
values, counts = np.unique(One_c_SVM_test, return_counts=True)
print(values)
print(counts)
print(counts[1]/(counts[0]+counts[1]))

#The result indicates that 40% of true-positives will be labelled as negatives

[-1  1]
[6 9]
0.6


### 2.3 Applying the trained one-class SVM model

In [113]:
#Apply the one class SVM model to the calgary data frame
One_c_SVM_result=One_class_SVM.predict(calgary_grouped_df.loc[:, calgary_grouped_df.columns != 'Neighborhood name'])
One_c_SVM_result

array([-1,  1, -1,  1,  1,  1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1])

In [115]:
#Add results to calgary df
calgary_df1['Result'] = One_c_SVM_result
calgary_df1.head()

,PostalCode,Neighborhood,Latitude,Longitude,Result
1,T3A,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12454,-114.14289,-1
2,T2B,"Forest Lawn, Dover, Erin Woods",51.02533,-113.97890,1
3,T3B,"Montgomery, Bowness, Silver Springs, Greenwood",51.08963,-114.19751,-1
4,T2C,"Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains",50.98122,-113.99786,1
5,T3C,"Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta",51.04492,-114.13070,1


In [118]:
#Get positive results only
calgary_df1_pos = calgary_df1.loc[calgary_df1['Result']==1]
calgary_df1_pos

,PostalCode,Neighborhood,Latitude,Longitude,Result
2,T2B,"Forest Lawn, Dover, Erin Woods",51.02533,-113.97890,1
4,T2C,"Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains",50.98122,-113.99786,1
5,T3C,"Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta",51.04492,-114.13070,1
6,T2E,"Bridgeland, Greenview, Zoo, YYC",51.07029,-114.04284,1
9,T3G,"Hawkwood, Arbour Lake, Citadel, Ranchlands, Royal Oak, Rocky Ridge",51.13818,-114.20157,1
17,T3L,"Tuscany, Scenic Acres",51.12323,-114.24007,1
22,T3P,Symons Valley,51.17748,-114.10508,1
28,T1Y,"Rundle, Whitehorn, Monterey Park",51.08058,-113.96087,1


### 2.4 Visualization of one-class SVM model results in a map

In [119]:
# libraries for map
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [123]:
# create map. The latitude and longitude of toronto were found on 
# https://www.latlong.net/place/calgary-ab-canada-29104.html
svm_map = folium.Map(location=[51.049999,  -114.066666], zoom_start=10.4)

#Add markers to map
for lat, lon in zip(calgary_df1_pos['Latitude'], calgary_df1_pos['Longitude']):
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(svm_map)
     
svm_map

# 3 Using Positive-Unlabelled data and SVM

## 3.1 Get and prepare the unlabelled data set

In [5]:
#Put the contents in a list
unlabelled_df_content = []

In [6]:
#Define a function that gets postal codes from sites with city and neighborhood in parenthesis
def getPostalCodes1(url, city_of_interest):
    html_data = requests.get(url).text
    soup = BeautifulSoup(html_data,"html5lib") 
    table = soup.find("table")
    for row in table.find_all("td") :
        cell = {}
        if row.span.text=='Not assigned':
            pass
    
        else:
            borough = row.span.text.split('(')[0]
            if borough== city_of_interest:
                example_labs_post_codes = example_labs_df.iloc[:,0].tolist()
                PostalCode=row.text[:3]
            
                if PostalCode in example_labs_post_codes:
                    pass
            
                else:
                    cell['PostalCode'] = PostalCode
                    cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
                    unlabelled_df_content.append(cell)
    
            else:
                pass

In [7]:
url_list1=['https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_K',
          'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_R']
cities=['Ottawa', 'Winnipeg']


In [8]:
#Ottawa and Winnipeg
getPostalCodes1(url=url_list1[0], city_of_interest=cities[0])
getPostalCodes1(url=url_list1[1], city_of_interest=cities[1])

In [9]:
#Define a function that gets postal codes from sites without cities named
def getPostalCodes2(url):
    html_data = requests.get(url).text
    soup = BeautifulSoup(html_data,"html5lib") 
    table = soup.find("table")
    for row in table.find_all("td") :
        cell = {}
        if row.span.text=='Not assigned':
            pass
    
        else:
            example_labs_post_codes = example_labs_df.iloc[:,0].tolist()
            PostalCode=row.text[1:4]
            if PostalCode in example_labs_post_codes:
                pass
            
            else:
                cell['PostalCode'] = PostalCode
                cell['Neighborhood'] = row.span.text
                unlabelled_df_content.append(cell)

In [10]:
#Montreal and Laval
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H'
getPostalCodes2(url)

In [11]:
#Define a function that gets postal codes from sites with neighborhoods in parathensis
def getPostalCodes3(url):
    html_data = requests.get(url).text
    soup = BeautifulSoup(html_data,"html5lib") 
    table = soup.find("table")
    for row in table.find_all("td") :
        cell = {}
        if row.span.text=='Not assigned':
            pass
    
        else:
            example_labs_post_codes = example_labs_df.iloc[:,0].tolist()
            PostalCode=row.text[1:4]
            if PostalCode in example_labs_post_codes:
                pass
            
            else:
                cell['PostalCode'] = PostalCode
                cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')   
                unlabelled_df_content.append(cell)

In [12]:
#Toronto
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
getPostalCodes3(url)

In [13]:
#Put together in a dataframe
unlabelled_df=pd.DataFrame(data=unlabelled_df_content)
print(unlabelled_df.shape)
print(len(set(unlabelled_df.iloc[:,0].tolist())))
unlabelled_df.head()

(256, 2)
256


,PostalCode,Neighborhood
0,K2A,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood"
1,K4A,Fallingbrook
2,K1B,"Blackburn Hamlet, Pine View, Sheffield Glen"
3,K2B,"Britannia,Whitehaven, Bayshore, Pinecrest"
4,K4B,Navan


In [19]:
#Unlabelled coordinates
unlabelled_coordinates_url ='https://raw.githubusercontent.com/Armando12pdf/Coursera_Capstone/main/Unlabelled%20locations%20coordinates.csv'
unlabelled_coordinates_df=pd.read_csv(unlabelled_coordinates_url)
unlabelled_coordinates_df.head()

,Postal code,Latitude,Longitude
0,K2A,45.38025,-75.76138
1,K4A,45.46734,-75.47799
2,K1B,45.42042,-75.59603
3,K2B,45.36172,-75.78945
4,K4B,45.41413,-75.40364


In [20]:
#Concatenate with respective latitude and longitude
unlabelled_df['Latitude'] = unlabelled_df.PostalCode.map(unlabelled_coordinates_df.set_index('Postal code')['Latitude'].to_dict())
unlabelled_df['Longitude'] = unlabelled_df.PostalCode.map(unlabelled_coordinates_df.set_index('Postal code')['Longitude'].to_dict())
print(unlabelled_df.shape)
unlabelled_df.head()

(256, 4)


,PostalCode,Neighborhood,Latitude,Longitude
0,K2A,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138
1,K4A,Fallingbrook,45.46734,-75.47799
2,K1B,"Blackburn Hamlet, Pine View, Sheffield Glen",45.42042,-75.59603
3,K2B,"Britannia,Whitehaven, Bayshore, Pinecrest",45.36172,-75.78945
4,K4B,Navan,45.41413,-75.40364


In [24]:
#Get venues for unlabelled neighborhoods
unlabelled_venues = getNearbyVenues(names=unlabelled_df['Neighborhood'],
                                   latitudes=unlabelled_df['Latitude'],
                                   longitudes=unlabelled_df['Longitude'])

Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood
Fallingbrook
Blackburn Hamlet, Pine View, Sheffield Glen
Britannia,Whitehaven, Bayshore, Pinecrest
Navan
Queensway, Copeland Park, Central Park, Bel Air,Carleton Heights
Cumberland
Queenswood
Eastern Nepean: Fisher Heights/ Parkwood Hills, Borden Farm, Pine Glen
Centrepointe, Meadowlands, City View, Craig Henry, Tangelwood, Grenfell Glen, Davidson Heights
Bells Corners, Arlington Woods/Redwood, Qualicum, Crystal Beach
Beacon Hill, Cyrville, Carson Grove
Barrhaven
Beaverbrook, South March
Katimavik-Hazeldean, Glen Cairn
Rockcliffe Park, New Edinburgh
Bridlewood
Manotick
Downtown
Greely
Dalhousie Ward
Fallowfield Village, Cedarhill Estates, Orchard Estates
The Glebe, Old Ottawa South, Old Ottawa East, Carleton University, Dow's Lake area
Stittsville
Blossom Park, Greenboro, Leitrim, Findlay Creek
Marchwood
Terry Fox, Palladium
Chapel Hill South, Blackburn
North March
South Gloucester
Civic Hospital, Island Park, Hintonburg, Me

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,Tillbury Park,45.378744,-75.759590,Park
1,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,Westboro Badminton Club,45.383524,-75.759266,Sports Club
2,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,McKellar Park,45.382858,-75.765691,Park
3,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,Jademark Technologies,45.383348,-75.756911,Electronics Store
4,Fallingbrook,45.46734,-75.47799,Marcel Lalande Park,45.469960,-75.475977,Park


In [25]:
print(unlabelled_venues.shape)
unlabelled_venues.head()

(5010, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,Tillbury Park,45.378744,-75.759590,Park
1,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,Westboro Badminton Club,45.383524,-75.759266,Sports Club
2,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,McKellar Park,45.382858,-75.765691,Park
3,"Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood",45.38025,-75.76138,Jademark Technologies,45.383348,-75.756911,Electronics Store
4,Fallingbrook,45.46734,-75.47799,Marcel Lalande Park,45.469960,-75.475977,Park


In [26]:
#One hot encoding of the unlabelled neighborhoods
unlab_onehot_df = pd.get_dummies(unlabelled_venues[['Venue Category']], prefix="", prefix_sep="")
unlab_onehot_df.insert(loc=0, column='Neighborhood name', value=unlabelled_venues['Neighborhood'].values.tolist()) 
print(unlab_onehot_df.shape)
unlab_onehot_df.head()

(5010, 347)


,Neighborhood name,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / 

In [52]:
#Regroup the venues of the example labs
unlab_grouped_df = unlab_onehot_df.groupby('Neighborhood name').mean().reset_index()
print(unlab_grouped_df.shape)
unlab_grouped_df.head()

(249, 347)


,Neighborhood name,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / 

## 3.2 Prepare the PU data set

In [58]:
#The postivie data set, unlabelled data, and calgary data set do not match in features
print('Before matching:')
print('Features in unlabelled data set: ' + str(len(unlab_grouped_df.columns)))
print('Features in positive data set: ' + str(len(labs_grouped_df.columns)))
print('Features in calgary data set: ' + str(len(calgary_grouped_df.columns)))

Before matching:
Features in unlabelled data set: 347
Features in positive data set: 213
Features in calgary data set: 102


In [85]:
#Make the postivie data set, unlabelled data, and calgary data set match in features
all_features =[]
all_features = unlab_grouped_df.columns.tolist() + labs_grouped_df.columns.tolist() + calgary_grouped_df.columns.tolist()

print('Total number of features is: ' + str(len(calgary_grouped_df.columns)+len(labs_grouped_df.columns)+len(unlab_grouped_df.columns)))
print('Number of features put together in a single list: ' + str(len(all_features)))


Total number of features is: 662
Number of features put together in a single list: 662


In [106]:
features_values, features_counts = np.unique(all_features, return_counts=True)

features_dict = {'features':features_values, 'counts':features_counts}
features_df1 = pd.DataFrame(features_dict)
features_df1.iloc[1:7,:]

,features,counts
1,Accessories Store,1
2,Adult Boutique,2
3,Afghan Restaurant,1
4,African Restaurant,1
5,Airport,2
6,Airport Lounge,1


In [105]:
features_df2=features_df1.loc[features_df1['counts']==3]
features_df2.head()

,features,counts
9,American Restaurant,3
13,Art Gallery,3
17,Asian Restaurant,3
18,Athletics & Sports,3
27,Bakery,3


In [114]:
common_features=features_df2['features'].values.tolist()
unlab_grouped_df2 = unlab_grouped_df[common_features]
labs_grouped_df2 = labs_grouped_df[common_features]
calgary_grouped_df2 = calgary_grouped_df[common_features]

print('Before matching:')
print('Features in unlabelled data set: ' + str(len(unlab_grouped_df2.columns)))
print('Features in positive data set: ' + str(len(labs_grouped_df2.columns)))
print('Features in calgary data set: ' + str(len(calgary_grouped_df2.columns)))

Before matching:
Features in unlabelled data set: 76
Features in positive data set: 76
Features in calgary data set: 76


In [97]:
#unlabelled_df_content

In [27]:
#Define a function that gets postal codes 
def getPostalCodes1(url, city_of_interest):
    html_data = requests.get(url).text
    soup = BeautifulSoup(html_data,"html5lib") 
    table = soup.find("table")
    for row in table.find_all("td") :
        cell = {}
        if row.span.text=='Not assigned':
            pass
    
        else:
            try:
                cell['PostalCode'] = row.text[:3]
                cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
                unlabelled_df_content.append(cell)
            
            
            except:
                try:
                    borough = row.span.text.split('(')[0]
                    if borough=='Not assigned':
                        print(borough)
                    else:
                        cell['PostalCode'] = row.text[:3]
                        cell['Neighborhood']=borough
                        unlabelled_df_content.append(cell)
                except:
                    pass

In [49]:
# Extract the rows of the example labs
example_labs_df = calgary_merged[calgary_merged['Neighborhood'].str.contains('lab', regex=False)]
print(example_labs_df.shape)
example_labs_df.head()

(13, 5)


,PostalCode,Neighborhood,Latitude,Longitude,Cluster Labels
31,R3P,example lab 1,49.83926,-97.20586,2
32,J8T,example lab 2,45.47843,-75.70476,2
33,K2P,example lab 3,45.41614,-75.69180,2
34,H4R,example lab 4,45.50834,-73.71150,2
35,H7L,example lab 5,45.60557,-73.78127,0


In [57]:
# Count the appearance of each cluster in the example labs dataframe
example_labs_df['Cluster Labels'].value_counts()

2    12
0     1
Name: Cluster Labels, dtype: int64

In [55]:
# Create a quick visual of the results 


AttributeError: 'Series' object has no attribute 'DataFrame'